<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/Calculating_parameters_with_formulas_linear_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 28.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=751578424989476e5f761f28679dbe5cd80a5009d6bdbfce3debab14ea4e9c3e
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [5]:
rd = sc.textFile("taxi-data-sorted-small.csv.bz2")

In [6]:
rd.take(3)

['07290D3599E7A0D62097A346EFCC1FB5,E7750A37CAB07D0DFF0AF7E3573AC141,2013-01-01 00:00:00,2013-01-01 00:02:00,120,0.44,-73.956528,40.716976,-73.962440,40.715008,CSH,3.50,0.50,0.50,0.00,0.00,4.50',
 '22D70BF00EEB0ADC83BA8177BB861991,3FF2709163DE7036FCAA4E5A3324E4BF,2013-01-01 00:02:00,2013-01-01 00:02:00,0,0.00,0.000000,0.000000,0.000000,0.000000,CSH,27.00,0.00,0.50,0.00,0.00,27.50',
 '0EC22AAF491A8BD91F279350C2B010FD,778C92B26AE78A9EBDF96B49C67E4007,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.71,-73.973145,40.752827,-73.965897,40.760445,CSH,4.00,0.50,0.50,0.00,0.00,5.00']

In [7]:
words = rd.map(lambda x: x.split(','))

In [8]:
words.count()

309206

In [12]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

In [13]:
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) !=0 and float(p[11]) !=0):
                return p

In [14]:
wordsCorr = words.filter(lambda x: correctRows(x))

In [15]:
wordsCorr.count()

307361

In [18]:
dist_fair = wordsCorr.map(lambda x: (float(x[5]), float(x[16])))

In [19]:
dist_fair.take(3)

[(0.44, 4.5), (0.71, 5.0), (0.48, 5.0)]

In [25]:
xsum_ysum = dist_fair.reduce(lambda x, y: ((x[0] + y[0]), (x[1] + y[1])))

In [28]:
nxy = dist_fair.map(lambda x: x[0] * x[1]).reduce(lambda x, y: x+y)

In [29]:
nxy

28839995.489599477

In [33]:
n_x_sqrd = dist_fair.map(lambda x: x[0]**2).reduce(lambda x, y: x+y)

In [34]:
n_x_sqrd

7797756.941200037

In [37]:
n = dist_fair.count()

In [38]:
m = (nxy*n - xsum_ysum[0]*xsum_ysum[1])/(n_x_sqrd*n - xsum_ysum[0]**2)

In [39]:
m

3.1274078593878807

In [40]:
b = (n_x_sqrd*n - xsum_ysum[0]*nxy) / (n*n_x_sqrd - (xsum_ysum[0])**2)

In [41]:
b

-19.5804992680945

Task - 2